## 1. Pré-processamento

O *dataset* original consistia em uma série temporal com **2.356.110 instâncias** e seis colunas principais (`date`, `state`, `name`, `code`, `cases`, `deaths`). Para atender aos objetivos de **aprendizado não supervisionado**, foi necessário transformar esses dados brutos em uma representação **descritiva e consolidada por município**.

#### Limpeza e Agregação dos Dados

Como cada município aparecia repetidamente ao longo do tempo, realizamos etapas de agregação para sintetizar seu perfil epidemiológico:

- **Tratamento dos dados:** remoção de valores nulos, registros em branco e colunas irrelevantes para a análise de *clusters*.
- **Engenharia de atributos (*feature engineering*):**
  - `total_cases`: total acumulado de casos por município.
  - `peak_cases`: maior número de casos registrados em um único dia, representando a intensidade do pico pandêmico local.

Essas transformações permitiram capturar tanto a magnitude quanto a dinâmica da disseminação da doença em cada localidade.

### Preparação para os Algoritmos de *Machine Learning*

Considerando que os algoritmos de *Machine Learning* operam exclusivamente sobre dados numéricos e são sensíveis à escala das variáveis, aplicamos as seguintes técnicas:

- **One-Hot Encoding:** conversão de variáveis categóricas em representações numéricas, possibilitando o seu uso nos modelos.
- **Normalização (`StandardScaler`):** padronização das variáveis numéricas.

#### &emsp; 1.1 Baixando o dataset

In [ ]:
# Intalação do gdown para pegar .csv bruto
%pip install gdown

Note: you may need to restart the kernel to use updated packages.


In [43]:
# Baixar o .csv bruto
!gdown --id "1sg9QK4g8QKCNgvfi6iNowcP75KNNkxUY" -O ./data/brazil_covid19_cities.csv

/mnt/dev/clones/project-unsupervised/.venv/lib/python3.13/site-packages/gdown/__main__.py:139: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1sg9QK4g8QKCNgvfi6iNowcP75KNNkxUY
From (redirected): https://drive.google.com/uc?id=1sg9QK4g8QKCNgvfi6iNowcP75KNNkxUY&confirm=t&uuid=dd11573c-0916-4784-b580-18ecf5d8a5c7
To: /mnt/dev/clones/project-unsupervised/Final/data/brazil_covid19_cities.csv
100%|████████████████████████████████████████| 106M/106M [00:59<00:00, 1.77MB/s]


In [44]:
# Importação do csv
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.neighbors import NearestNeighbors
import numpy as np
import umap
df = pd.read_csv('./data/brazil_covid19_cities.csv')

#### &emsp; 1.2 Modelando os dados 

In [ ]:
# Renomear coluna name para city
df = df.rename(columns={'name': 'city'})

In [ ]:
# Transforma a coluna `date` em um objeto DateTime
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['city', 'date'])

In [ ]:
# Garantir que casos e mortes são numéricos
df['cases'] = pd.to_numeric(df['cases'], errors='coerce')
df['deaths'] = pd.to_numeric(df['deaths'], errors='coerce')

df = df.dropna(subset=['city', 'cases', 'deaths'])

In [ ]:
# Adição de variáveis por cidade e estado
features_city = df.groupby(['city', 'state']).agg({
  'cases': ['max', 'mean', 'std'],
  'deaths': ['max', 'mean', 'std'],
  'date': 'count'
})

features_city.columns = [
  'total_cases',
  'mean_cases',
  'std_cases',
  'total_deaths',
  'mean_deaths',
  'std_deaths',
  'days_recorded'
]

features_city.head()

In [ ]:
# Adição de coluna `death_rate` (taxa de mortalidade)
features_city['death_rate'] = (
    features_city['total_deaths'] / features_city['total_cases']
)


In [ ]:
# Crescimento dos casos
df['new_cases'] = df.groupby(['city', 'state'])['cases'].diff().fillna(0)

growth = df.groupby(['city', 'state'])['new_cases'].mean().rename('mean_daily_growth')
features_city = features_city.join(growth)

In [ ]:
# Dia do primeiro caso
first_case = df[df['cases'] > 0].groupby(['city','state'])['date'].min()
first_case = (first_case - df['date'].min()).dt.days
first_case = first_case.rename('days_until_first_case')

features_city = features_city.join(first_case)

In [ ]:
# Remoção de registros com dados faltosos
features_city = features_city.dropna()

In [ ]:
# Reset de índices
features_city = features_city.reset_index()

In [ ]:
# Remoção do do campo `city`
features_city.drop('city', axis=1, inplace=True)

In [ ]:
# Aplicando one-hot encode no campo `state`
features_city = pd.get_dummies(features_city, columns=['state'])

In [ ]:
# Finalização do DataFrame tratado
df = features_city.copy()

In [ ]:
df

## 2. Redução de Dimensionalidade


#### &emsp; 2.1 Aplicação de PCA

#### &emsp;  2.2 Aplicação de t-SNE

#### &emsp;  2.3 Aplicação de UMAP

## 3. Clusterização

#### &emsp;  3.1 Aplicação de K-Means

#### &emsp;  3.2 Aplicação de DBSCAN

## 4. Discussão de resultados